# GLM Demo: univariate stimulus, biophysical dynamics

This is a demo notebook for the GLM estimation algorithm. Here we simulate data using the biophysical dynamical cascade model, and then try to estimate parameters with the GLM. The GLM does not have a membrane, so the estimated RF should be the convolution of the input kernel ($k1$) with the membrane kernel ($k2$) which is approximately an exponential decay with time constant $\tau_m$. However, what can happen is that the kernel gets shifted to shorter lags for the phasic model, which is consistent with what Chen and Meliza (2018) found for the coherence between an input current and the spiking output - the dynamics act as a bandpass filter.

This notebook also introduces the use of a raised cosine basis set to reduce the number of parameters in the kernel estimate.

In [ ]:
from __future__ import print_function, division
import sys
import imp
import os
import numpy as np
import scipy as sp
import scipy.signal as sps
import quickspikes as qs

import mat_neuron._model as mat
from dstrf import strf, mle, filters
import spyks.core as spkc

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
model_dt = 0.5
stim_dt = 10.0

# decay times for exponential adaptation basis set
ataus = np.asarray([10, 200], dtype='d')

# convolution kernel
ntau = 60
upsample = int(stim_dt / model_dt)
ntbas = 8

k1, kt = filters.gammadiff(ntau * stim_dt / 32, ntau * stim_dt / 16, 10.0, ntau * stim_dt, stim_dt)
plt.plot(kt, k1)

# raised-cosine basis functions
kcosbas = strf.cosbasis(ntau, ntbas)
ntbas = kcosbas.shape[1]
k1c = strf.to_basis(k1, kcosbas)

## Generate the stimulus and response

We are also using gaussian white noise here.

In [ ]:
# generating spikes with biocm
# this needs to be adjusted on a per model basis. posp ~ 2.0; phasic ~ 10
# model_name = "biocm_phasic"
# current_scaling = 9.0
model_name = "biocm_tonic"
current_scaling = 4.0
# model_name = "pospischil_sm"
# current_scaling = 2.0

trial_noise_sd = 2.0
spike_thresh = -20
dt_rise_time = int(1.0 / model_dt)
modelpath = "../../models"
pymodel = spkc.load_model(os.path.join(modelpath, model_name + ".yml"))
biocm_params = spkc.to_array(pymodel["parameters"])
biocm_state0 = spkc.to_array(pymodel["state"])
biocm_model = spkc.load_module(pymodel, modelpath)
    
def filter_stimulus(S, k1):
    return np.convolve(S, k1, mode="full")[:S.size]

def generate_spikes(I, noise_sd, dt, upsample):
    from scipy.signal import resample
    I_noise = np.random.randn(I.size) * noise_sd
    I = current_scaling * (I + I_noise)
    #I_resamp = sps.resample(I + I_noise, I.size * upsample)
    X = biocm_model.integrate(biocm_params, biocm_state0, I, stim_dt, model_dt)
    det = qs.detector(spike_thresh, dt_rise_time)
    return I, X[:, 0], det(X[:, 0])

In [ ]:
# data parameters
duration = 100000
n_bins = int(duration / model_dt)
n_frames = n_bins // upsample
n_assim = 10
n_test = 10

# generate data to fit
np.random.seed(1)
mat.random_seed(1)
data = []
stim = np.random.randn(n_frames)
stim[:100] = 0
        
I = filter_stimulus(stim, k1)
for i in range(n_assim):
    In, V, spike_t = generate_spikes(I, trial_noise_sd, model_dt, upsample)
    spike_v = np.zeros(V.size, 'i')
    spike_v[spike_t] = 1
    H = mat.adaptation(spike_v, ataus, model_dt)
    d = {"H": H,
         "duration": duration,
         "spike_t": np.asarray(spike_t), 
         "spike_v": spike_v,
        }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(9, 4))
t_stim = np.linspace(0, duration, stim.size)
t_voltage = np.linspace(0, duration, V.size)
axes[0].set_title("Simulated {} response".format(model_name))
axes[0].plot(t_stim, stim)
axes[1].plot(t_stim, I, t_voltage, V)
for i, d in enumerate(data):
    axes[2].vlines(d["spike_t"] * model_dt, i, i + 0.5)
for ax in axes:
    ax.set_xlim(0, 8000)
print("spikes: {}; rate: {} / dt".format(np.mean([d["spike_t"].size for d in data]), 
                                         np.mean([d["spike_t"].size / d["duration"] for d in data])))

## Estimation

The theano code for setting up maximum likelihood estimation has been factored out into the `mle` package.

**Note**: The following cell sometimes generates an error the first time you run it. Just run it again if that happens.

In [ ]:
# initial guess of parameters using regularized ML
ntbas = 8
kcosbas = strf.cosbasis(ntau, ntbas)
spike_v = np.stack([d["spike_v"] for d in assim_data], axis=1)
spike_h = np.stack([d["H"] for d in assim_data], axis=2)
try:
    mlest = mle.mat(stim, kcosbas, spike_v, spike_h, stim_dt, model_dt, nlin="exp")
except TypeError:
    mlest = mle.mat(stim, kcosbas, spike_v, spike_h, stim_dt, model_dt, nlin="exp")
%time w0 = mlest.estimate(reg_alpha=1.0)

In [ ]:
print("MLE rate and adaptation parameters:", w0[:3])
rf_ml = strf.from_basis(w0[3:], kcosbas)[::-1]
plt.plot(k1, label="kernel")

# there is an expected shift due to the filtering properties of the membrane
km, kmt = filters.exponential(46, 1.0, ntau * stim_dt, stim_dt)
kconv = np.convolve(km, k1, mode="full")[:km.size]
kconv *= k1.max() / kconv.max()
plt.plot(kconv, label="expected")

plt.plot(rf_ml * k1.max() / rf_ml.max(), label="MLE")
plt.legend()